In [1]:
from tqdm import tqdm
import numpy as np
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd()+'/corpus')

In [3]:
args_in = '--device gpu0 '\
        '--modelTag udc_2l_lr002_dr09_hid256_emb256_len50_vocab10000 '\
        '--hiddenSize 256 --embeddingSize 256 '\
        '--vocabularySize 10000 --maxLength 50 '\
        '--learningRate 0.002 --dropout 0.9 '\
        '--rootDir C:\\Users\\reade\\Documents\\lecture3 '\
        '--datasetTag round3_7 --corpus ubuntu'.split()
        

### 恢复（restore）best_model的参数
**注意**

`ckpt_file = 'your_filepath\\model_dir\\best_model.ckpt'`

而不是

`ckpt_file = 'your_filepath\\model_dir\\best_model.ckpt.data-00000-of-00001'`

即便后者是你在文件夹里面能够看到的真实文件而前者只是一些文件的前缀

`best_model.ckpt.data-00000-of-00001`
`best_model.ckpt.meta`
`best_model.ckpt.index`

        

### 输入validation数据，运行evaluation任务



In [4]:
"""Changed from Rankbot, used for inference
"""
import os  # 文件管理
import tensorflow as tf
from textdata import (TextData, RankTextData)
from ranker import Ranker
from rankbot import Rankbot

def ranking(args=None, batch_valid=None):
    args = Rankbot.parseArgs(args)
    if not args.rootDir:
        args.rootDir = os.getcwd()

    # 搭建/restore 模型
    graph = tf.Graph()

    with graph.as_default():
        with tf.name_scope('training'):
                model_train = Ranker(args, is_training = True)

        tf.get_variable_scope().reuse_variables()
        with tf.name_scope('validation'):
            model_valid = Ranker(args, is_training = False)

        with tf.name_scope('evluation'):
            model_test = Ranker(args, is_training = False)
            ckpt_model_saver = tf.train.Saver(name = 'checkpoint_model_saver')
            best_model_saver = tf.train.Saver(name = 'best_model_saver')


        # 运行session
        # allow_soft_placement = True: 当设置为使用GPU而实际上没有GPU的时候，允许使用其他设备运行。
        sess = tf.Session(config=tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False)
        )
        # 恢复指定模型的参数
        ckpt_file = 'C:\\Users\\reade\\Documents\\lecture3\\save\\model-udc_2l_lr002_dr09_hid256_emb256_len50_vocab10000\\best_model.ckpt'            
        best_model_saver.restore(sess, ckpt_file)
        valid_acc = [0, 0, 0]
        for nextEvalBatch in tqdm(batches_valid):
            ops, feedDict = model_valid.step(nextEvalBatch)
            loss, eval_summaries = sess.run(ops, feedDict)
            break
            for i in range(3):
                valid_acc[i] += loss[i]/len(batches_valid)
    return loss, nextEvalBatch, valid_acc

#### 一个class staticmethod的案例: `args = Rankbot.parseArgs(args)`
#### 产生minibatch形式的validation dataset

In [5]:
args = Rankbot.parseArgs(args_in)
evalData = RankTextData(args)
batches_valid = evalData.getValidBatches()

filteredSamplesPath:C:\Users\reade\Documents\lecture3\data/samples\dataset-ubuntu-round3_7-length50-filter1-vocabSize10000.pkl
Loading dataset from C:\Users\reade\Documents\lecture3\data/samples\dataset-ubuntu-round3_7-length50-filter1-vocabSize10000.pkl
Loaded ubuntu: 10004 words, 1079144 training QA samples


### 计算validation数据上的ranking排名，并返回一个minibatch的输入，真实回复和negative回复的score

In [6]:
loss, batch, acc = ranking(args_in, batches_valid)

模型初始化...
模型初始化...
模型初始化...
INFO:tensorflow:Restoring parameters from C:\Users\reade\Documents\lecture3\save\model-udc_2l_lr002_dr09_hid256_emb256_len50_vocab10000\best_model.ckpt


  0%|                                                                                         | 0/4216 [00:00<?, ?it/s]


In [26]:
print(acc)

[0.53538831010941268, 0.76440312089911577, 0.85885329242002106]


In [7]:
rank1, rank3, rank5, logits = loss
logits.shape

(32, 20)

In [8]:
queries = []
for i in range(len(batch.query_seqs)):
    queries.append(' '.join([evalData.id2word[x] for x in batch.query_seqs[i] if x != 0]) )

In [9]:
batch.response_seqs
responses = []
for i in range(len(batch.response_seqs)):
    responses.append(' '.join([evalData.id2word[x] for x in batch.response_seqs[i] if x != 0]) )

In [10]:
max_idx = np.argmax(logits, axis = 1)
true_pred = []
false_pred = []

for i in range(len(max_idx)):
    if max_idx[i]==0:
        true_pred.append(i)
    else:
        false_pred.append(i)

## 一些正确地将真实恢复排在第一位的例子

In [11]:
eg1 = true_pred[0]
print('query sentence:')
print('\t%s' % queries[eg1])
print('true response:')
print('\t%s' % responses[eg1*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg1])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg1*20 + order[-i-1]])

query sentence:
	packages.ubuntu.com
true response:
	thanks


all responses sorted according to scores
response ranked 0
	thanks
response ranked 1
	yes _eou_ it is not mounting
response ranked 2
	ahh , let me help you
response ranked 3
	it 's just the updates , that leave the old kernels there
response ranked 4
	yes i see . well , first thing.. i 'm gon na reboot .
response ranked 5
	called sudo my friend
response ranked 6
	did you order them the day before ? wow
response ranked 7
	i do n't think so
response ranked 8
	they are installed via the ubuntu-desktop meta package i think _eou_ but do n't quote me on that
response ranked 9
	i want to create a program using c or c++ code and i do n't know how to write and run it on kdevelop
response ranked 10
	se files i want to share are already mounted to home and they are the files i wish to share but i read that it is good practice to have shares located at srv samba so i thought i could also mount them to that location ? _eou_
response rank

In [12]:
eg2 = true_pred[1]
print('query sentence:')
print('\t%s' % queries[eg2])
print('true response:')
print('\t%s' % responses[eg2*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg2])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg2*20 + order[-i-1]])

query sentence:
	does anyone else notice that firefox uses a lot of cpu ?
true response:
	firefox does that . especially if you 're low on ram


all responses sorted according to scores
response ranked 0
	firefox does that . especially if you 're low on ram
response ranked 1
	no . _eou_ probably
response ranked 2
	no _eou_ it is safer
response ranked 3
	do your question
response ranked 4
	neither , keep the offtopic chat somewhere else
response ranked 5
	which machine ?
response ranked 6
	you can do r to take it into raw mode and then try whatever
response ranked 7
	, are you chinese ?
response ranked 8
	hmm
response ranked 9
	http : www kernel org _eou_ less will display it in a format that you can scroll through _eou_ then follow erusul 's suggestion and usr lpr
response ranked 10
	and who are you , if i might want to know ?
response ranked 11
	hmmm ... reiserfs ... ext3 ?
response ranked 12
	oh . then it 's just `` apt-get upgrade ''
response ranked 13
	so you want a webserver ? _eo

In [13]:
eg3 = true_pred[2]
print('query sentence:')
print('\t%s' % queries[eg3])
print('true response:')
print('\t%s' % responses[eg3*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg3])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg3*20 + order[-i-1]])

query sentence:
	i ca n't play audio files using audio players . can somebody help me fix this ? _eou_ <unknown> : yup but i have played the files before . _eou_ <unknown> : i have played it before using amarok . _eou_ <unknown> : yup .
true response:
	what version are you on _eou_ maybe you need to download the plug in for amarok _eou_ have you upgraded


all responses sorted according to scores
response ranked 0
	what version are you on _eou_ maybe you need to download the plug in for amarok _eou_ have you upgraded
response ranked 1
	sounds like stormx2 has it solved
response ranked 2
	please do n't flood ; use http : paste ubuntu com to paste ; do n't use enter as punctuation .
response ranked 3
	: (
response ranked 4
	you may ask
response ranked 5
	yes , it 's free .
response ranked 6
	you need to enable multiverse in sources.list
response ranked 7
	sorry !
response ranked 8
	enjoy : https : help ubuntu com 9 04 serverguide c index html
response ranked 9
	i 'm on windows
response r

## 一些将真实回复排在第二位或者更靠后的例子

In [57]:
eg4 = false_pred[0]
print('query sentence:')
print('\t%s' % queries[eg4])
print('true response:')
print('\t%s' % responses[eg4*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg4])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg4*20 + order[-i-1]])

query sentence:
	hi all _eou_ what 's going on with # <unknown> . i cant seem so get connected
true response:
	perhaps they decided to be # philosophy instead ?


all responses sorted according to scores
response ranked 0
	we 'll someone here has to have a network printer , the gnome they used in feisty had a very easy interface , then they changed it
response ranked 1
	perhaps they decided to be # philosophy instead ?
response ranked 2
	there 's also citrix if you have a citrix server in house _eou_ emacs dude
response ranked 3
	o.o
response ranked 4
	hit alt+f2 and run gnome-system-monitor to get the `` task manager ''
response ranked 5
	nalioth yes to local host works
response ranked 6
	do n't understand , where did you refresh ?
response ranked 7
	language
response ranked 8
	have you used it with the accelerator patch ?
response ranked 9
	i ca n't understand you
response ranked 10
	! broadcom is a `` factoid '' i sent you , for some help with a particular bc wireless , sorry if you

In [58]:
eg5 = false_pred[1]
print('query sentence:')
print('\t%s' % queries[eg5])
print('true response:')
print('\t%s' % responses[eg5*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg5])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg5*20 + order[-i-1]])

query sentence:
	i ca n't play audio files using audio players . can somebody help me fix this ? _eou_ <unknown> : yup but i have played the files before . _eou_ <unknown> : i have played it before using amarok . _eou_ <unknown> : yup .
true response:
	what version are you on _eou_ maybe you need to download the plug in for amarok _eou_ have you upgraded


all responses sorted according to scores
response ranked 0
	vlc does n't play wmv 's =
response ranked 1
	what version are you on _eou_ maybe you need to download the plug in for amarok _eou_ have you upgraded
response ranked 2
	define advanced
response ranked 3
	start with # kubuntu perhaps ? ask there _eou_ try join # kubuntu
response ranked 4
	if you keep tryin ' , you 'll succeed : )
response ranked 5
	well in windows it works fine
response ranked 6
	yes
response ranked 7
	only if you home is a seperate partition
response ranked 8
	no . kubuntu is based on ubuntu , but is not the same , so you do not need the kubuntu package
resp

In [15]:
eg6 = false_pred[2]
print('query sentence:')
print('\t%s' % queries[eg6])
print('true response:')
print('\t%s' % responses[eg6*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg6])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg6*20 + order[-i-1]])

query sentence:
	for kubuntu ? _eou_ you are referring to a ppa perhaps
true response:
	you mean https : launchpad net ubuntu ?


all responses sorted according to scores
response ranked 0
	i did it on
response ranked 1
	oops , apt-get dist-upgrade , gotcha ! i 'll probably do it from a text console then if it does n't matter - who knows ? ; )
response ranked 2
	you mean https : launchpad net ubuntu ?
response ranked 3
	i really have no idea what the question is..
response ranked 4
	the last
response ranked 5
	yes , that will do it
response ranked 6
	it 's in the repos _eou_ the firefox plugin does n't work ?
response ranked 7
	thanks : ) been there but there is only 11 people on channel , that is when i 've tried here
response ranked 8
	packages are simply .deb files . you can transfer them as any file .
response ranked 9
	indicator applet
response ranked 10
	ok thanks mine freezes up
response ranked 11
	no lame questions we are all here for one another : d
response ranked 12
	i have 

In [14]:
eg7 = false_pred[3]
print('query sentence:')
print('\t%s' % queries[eg7])
print('true response:')
print('\t%s' % responses[eg7*20])
print('\n')
print('all responses sorted according to scores')

order = np.argsort(logits[eg7])
for i in range(20):
    print('response ranked %d' % i)
    print('\t%s' % responses[eg7*20 + order[-i-1]])

query sentence:
	lol _eou_ <unknown> _eou_ - %
true response:
	hi , how can we help you today ?


all responses sorted according to scores
response ranked 0
	please do n't flood ; use http : paste ubuntu com to paste ; do n't use enter as punctuation . _eou_ please do n't flood ; use http : paste ubuntu com to paste ; do n't use enter as punctuation . _eou_
response ranked 1
	yo
response ranked 2
	hi , how can we help you today ?
response ranked 3
	:3
response ranked 4
	i know you know that . i was n't being entirely serious
response ranked 5
	use msg nickserv help and talk to the bot .
response ranked 6
	please do _eou_ i gave you one _eou_ your on the minimum required ram
response ranked 7
	have you searched ubuntuforums ? lots of sound troubleshooting there .
response ranked 8
	no , generally it sets up both ipv4 and ipv6
response ranked 9
	do you have universe enabled
response ranked 10
	thanks
response ranked 11
	afaik that 's correct .
response ranked 12
	i dont know . it happens

## UDC论文上的案例分析